In [ ]:
from gensim.models.keyedvectors import FastTextKeyedVectors

import pandas as pd
from os import listdir
from os.path import isfile, join

import progressbar
import time

In [ ]:
fasttext = FastTextKeyedVectors.load("./FASTTEXT//araneum_none_fasttextcbow_300_5_2018.model")

In [ ]:
food_word = ['кушать','покушать','еда','пища','блюдо','суп','пир','питье',
             'выпивка' ,'закуска','жратва','угощение','лакомство','ужин','завтрак','напиток',
            'щи','каша','борщ','похлебка','свинина','говядина','пирог','квашеный','котлета','мясо','баранина','курятина',
            'овощ', 'фрукт']

def detect_food(line):
"""проходим по каждому слову строки, если есть слово близкое к еде, то вовзращаем соответствующую отметку"""
    try:
        words = line.split()
    except:
        return False, None
    for word in words:
        for food in  food_word:
            try:
                sim = fasttext.similarity(word, food)
                if(sim > 0.8):
                    print(line, ":",word,  food, sim)
                    food_stc = [line, word, food]
                    return True, food_stc
            except:
                pass
                
    return False, None

In [ ]:
def get_food_dialogues(cleaned_text, lemm_text, debug = False):

    bar = progressbar.ProgressBar(maxval=len(lemm_text),
                                  widgets=[progressbar.Bar('=', '[', ']'), ' ', progressbar.Percentage()])
    bar.start()
    lines = 0

    dialogues_phrases = []
    current_phrase = []
    similarity_dict = []
    add_response_next_iter = False

    ITERATION = 0

    stop = 0

    for line, orig_line in zip(lemm_text, cleaned_text):
        #print(line, orig_line, ITERATION)
        if(add_response_next_iter):
            if(debug): print("SECOND:", orig_line, ITERATION)
            current_phrase.append(orig_line)
            dialogues_phrases.append(current_phrase)
            assert (len(current_phrase) == 2)
            add_response_next_iter = False
            current_phrase = []
            
        
        add_response_next_iter, similarity = detect_food(line)

        if(similarity):
            if(debug):
                print(similarity, ITERATION)
                print(line, orig_line, ITERATION)
            similarity_dict.append(similarity)

        if(add_response_next_iter):
            if(debug): print("FIRST:", orig_line, ITERATION)
            current_phrase.append(orig_line)
            assert (len(current_phrase) == 1)

        ITERATION += 1


        lines += 1
        bar.update(lines)
        time.sleep(0.1)
        
    return dialogues_phrases

In [ ]:
all_subs_dialogues = []
for sub_df in listdir("./sub/proessed_subs/"):
    path_to_df = join("./sub/proessed_subs/",sub_df)
    print("NOW PROCESSING",path_to_df)
    try:
        df = pd.read_csv(path_to_df, sep='\t', encoding='cp1251')
        film_phrases = get_food_dialogues(df['cleaned_text'], df['lemm_text'], debug = True)
        all_subs_dialogues.extend(film_phrases)
    except:
        print("FILE_READING_FAILED")
        pass

In [ ]:
def save_dialogues(dialogues, line_number):
    fist_phrase = [phrase[0] for phrase in dialogues]
    reply = [phrase[1] for phrase in dialogues]
    dialogues_df = pd.DataFrame(
    {'fist_phrase': fist_phrase,
     'reply': reply,
    })
    df_name = "dialogues_from_subs.csv"
    dialogues_df.to_csv(df_name, sep='\t', encoding='utf-8', index=False)

In [ ]:
save_dialogues(all_subs_dialogues, "all_rest")